In [1]:
import pandas as pd 
import datetime

In [2]:
tripleg = pd.read_csv('triplegs.csv')

In [3]:
food_df = pd.read_csv('200m_food_related_stop.csv', index_col=0)

In [4]:
food_df.drop(columns=['id', 'finished_at'], inplace=True)
food_df

,user_id,started_at,lat,lon,retail_id,retail_lat,retail_lon
0,2,2022-09-10 02:09:19-04:00,29.655040,-81.657860,145696,29.655165,-81.656845
1,2,2022-09-26 02:07:17-04:00,29.655035,-81.657880,145696,29.655165,-81.656845
2,3,2022-09-05 20:01:05-04:00,29.651755,-82.339455,70229,29.653463,-82.339195
3,3,2022-09-07 20:02:10-04:00,29.651742,-82.339464,70229,29.653463,-82.339195
4,3,2022-09-08 20:01:10-04:00,29.651755,-82.339455,70229,29.653463,-82.339195
...,...,...,...,...,...,...,...
54533,34471,2022-10-22 02:50:59-04:00,29.407710,-82.248080,18330,29.406757,-82.248161
54534,34471,2022-10-30 01:31:03-04:00,30.002802,-82.597425,30767,30.001783,-82.597435
54535,34472,2022-10-26 23:04:35-04:00,30.322587,-81.557411,58188,30.321955,-81.556618
54536,34481,2022-10-25 20:52:41-04:00,29.651005,-82.330360,52511,29.650291,-82.330154


In [5]:
tripleg.drop(columns=['id'], inplace=True)
tripleg

,user_id,started_at,finished_at,geom
0,0,2022-09-06 13:11:03-04:00,2022-09-06 13:11:54-04:00,LINESTRING (-82.0707699999999960 29.1930100000...
1,0,2022-09-09 00:59:39-04:00,2022-09-09 00:59:49-04:00,LINESTRING (-82.1166076699999934 29.1995639800...
2,1,2022-09-06 09:15:08-04:00,2022-09-06 09:15:58-04:00,LINESTRING (-81.6775345799999997 29.6550393099...
3,1,2022-09-06 21:34:07-04:00,2022-09-06 21:34:57-04:00,LINESTRING (-81.6840577100000047 29.6517777400...
4,1,2022-09-07 00:05:48-04:00,2022-09-07 00:09:57-04:00,LINESTRING (-81.6582870000000014 29.6534939999...
...,...,...,...,...
896148,25067,2022-10-27 01:06:08-04:00,2022-10-27 01:09:53-04:00,LINESTRING (-81.6119800000000026 30.3824900000...
896149,25068,2022-10-31 02:06:17-04:00,2022-10-31 02:07:50-04:00,LINESTRING (-81.7572399999999959 30.3360599999...
896150,25069,2022-10-26 08:25:34-04:00,2022-10-26 08:25:48-04:00,LINESTRING (-82.0980799999999959 29.9914400000...
896151,25069,2022-10-29 20:39:33-04:00,2022-10-29 20:39:40-04:00,LINESTRING (-82.2332099999999997 29.3812399999...


In [6]:
food_df['started_at'] = pd.to_datetime(food_df['started_at'])
tripleg['finished_at'] = pd.to_datetime(tripleg['finished_at'])

In [7]:
ID_list=food_df['user_id'].unique()
len(ID_list)

4286

In [8]:
time_window = datetime.timedelta(minutes=15)

miss = 0
check_len = 0
data_to_append_list = []
for id in ID_list:
    related_food_by_user = food_df[food_df['user_id'] == id]
    related_tripled_by_user = tripleg[tripleg['user_id'] == id]
    if len(related_tripled_by_user) == 0: 
        miss += 1
        continue
    
    for food_related_row in related_food_by_user.itertuples():
        tripled_found = related_tripled_by_user[
            (related_tripled_by_user['finished_at'] > food_related_row[2] - time_window) &
            (related_tripled_by_user['finished_at'] <= food_related_row[2])
        ]
        #print(len(tripled_found))
        if len(tripled_found) == 0: continue #did not found related trip
        #check if there if repeat data
        if len(tripled_found) > 1: 
            check_len += 1
            tripled_found = tripled_found.nlargest(1, 'finished_at')
        
        data_to_append = {'user_id': id, 'trip_started_at': tripled_found.iloc[0]['started_at'],
                'trip_finished_at': tripled_found.iloc[0]['finished_at'],
                'trip':tripled_found.iloc[0]['geom'],
                'stop_lat': food_related_row[3], 'stop_lon': food_related_row[4],
                'retail_id': food_related_row[5], 'retail_lat': food_related_row[6], 'retail_lon': food_related_row[7]}
        
        data_to_append_list.append(data_to_append)
        

trip_df = pd.DataFrame(data_to_append_list, columns=['user_id', 'trip_started_at', 'trip_finished_at', 'trip', 'stop_lat', 'stop_lon', 'retail_id', 'retail_lat', 'retail_lon'])

In [9]:
trip_df

,user_id,trip_started_at,trip_finished_at,trip,stop_lat,stop_lon,retail_id,retail_lat,retail_lon
0,14,2022-09-09 05:43:42-04:00,2022-09-09 06:32:23-04:00,LINESTRING (-81.4952796000000035 30.3301333000...,29.141500,-82.442730,208730,29.140825,-82.442841
1,241,2022-09-06 14:01:17-04:00,2022-09-06 14:02:06-04:00,LINESTRING (-81.7395999999999958 30.1065000000...,30.185342,-82.599069,70890,30.184486,-82.599495
2,241,2022-09-09 14:01:21-04:00,2022-09-09 14:02:10-04:00,LINESTRING (-81.7395999999999958 30.1065000000...,30.185337,-82.599073,70890,30.184486,-82.599495
3,241,2022-09-28 15:22:11-04:00,2022-09-28 15:22:57-04:00,LINESTRING (-81.7116093599999971 30.0999426800...,30.178748,-82.666830,55797,30.178915,-82.665512
4,241,2022-09-29 12:59:32-04:00,2022-09-29 12:59:41-04:00,LINESTRING (-81.8552200000000028 30.0794600000...,30.183147,-82.639468,118196,30.181879,-82.639214
...,...,...,...,...,...,...,...,...,...
846,21535,2022-10-09 17:44:44-04:00,2022-10-09 17:46:12-04:00,LINESTRING (-81.6074189999999930 30.2525469999...,29.630327,-82.422462,14186,29.630577,-82.422417
847,21535,2022-10-10 21:34:07-04:00,2022-10-10 21:34:26-04:00,LINESTRING (-81.6479099999999960 30.2940199999...,29.702365,-82.390387,62572,29.701960,-82.388916
848,21535,2022-10-10 21:34:07-04:00,2022-10-10 21:34:26-04:00,LINESTRING (-81.6479099999999960 30.2940199999...,29.703439,-82.388177,62572,29.701960,-82.388916
849,21535,2022-10-14 18:18:34-04:00,2022-10-14 18:19:30-04:00,LINESTRING (-81.6341199999999958 30.2875340000...,29.618778,-82.383662,17112,29.618984,-82.383125


In [10]:
print(len(trip_df['user_id'].unique()))

225


In [11]:
trip_df.to_csv('food_related_trip.csv')